In [1]:
!pip install transformers
!pip install numpy==1.26.0
!pip install tensorflow[and-cuda]
!pip install torch torchvision torchaudio
!pip install --upgrade ipywidgets
!pip install tf-keras
!pip install pandas
!pip install scikit-learn
!pip install torchinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 23.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.19 requires numpy<2,>=1.26.4; python_version < "3.12", but you have numpy 1.26.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:0

In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## Data Preprocessing

In [1]:
import pandas as pd

In [2]:
train_data = pd.read_csv("data/train_data.csv")
test_data = pd.read_csv("data/test_data.csv")

In [3]:
def convert_lbl_to_int(label):
    if label.lower() == "positive":
        return 1
    if label.lower() == "negative":
        return 0
    return -1

## Baseline Model

In [4]:
import numpy as np
print(np.__version__)
from transformers import BertModel, BertTokenizer, BertForSequenceClassification
from transformers import pipeline
from sklearn.metrics import f1_score
from torchinfo import summary

1.26.0


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2025-02-24 09:52:30.220609: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740390750.243675   47148 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740390750.250726   47148 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-24 09:52:30.276708: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable

In [5]:
def get_prompt(review):
  with open("baseline_prompt.txt", "r") as file:
    prompt = file.read()
  prompt = prompt.replace("<REVIEW>", review)
  return prompt

print(get_prompt("Test Prompt"))

Given a review, classify it as positive or negative sentiment.
Review: Test Prompt
This review has a [MASK] sentiment.



In [6]:
# baseline_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# print(inputs)
# with torch.no_grad():
#     logits = baseline_model(**inputs).logits
#     print(logits.shape)

# predicted_class_id = logits.argmax().item()
# print(baseline_model.config.id2label[predicted_class_id])

# summary(baseline_model)

In [7]:
# Load the BERT model and tokenizer
baseline_model = BertModel.from_pretrained("bert-base-uncased")
baseline_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
baseline_unmasker = pipeline('fill-mask', model='bert-base-uncased')

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

In [8]:
def predict(review, unmasker):
  prompt = get_prompt(review)
  predictions = unmasker(prompt)
  for prediction in predictions:
    if(prediction['token_str'] == 'negative' or prediction['token_str'] == 'positive'):
      return prediction['token_str']

  return "<NOT FOUND>"

In [50]:
def predict_baseline(df, model, tokenizer, unmasker):
    predictions = []
    labels = []
    for idx, row in df.iterrows():
        input_text = row['Review']
        prompt = get_prompt(input_text)
        prediction = predict(input_text, unmasker)
        predictions.append(convert_lbl_to_int(prediction))
        labels.append(convert_lbl_to_int(row['Sentiment']))

    return predictions, labels

predictions, labels = predict_baseline(train_data, baseline_model, baseline_tokenizer, baseline_unmasker)

In [49]:
print(labels)
print(predictions)

[0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, -1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [51]:
values, counts = np.unique(np.array(predictions), return_counts=True)

for v, c in zip(values, counts):
    print(f"Value: {v}, Count: {c}")

Value: -1, Count: 149
Value: 0, Count: 581
Value: 1, Count: 7270


In [52]:
print( f1_score(labels, predictions, average='micro'))

0.556375


## Probe Models

In [9]:
import torch.nn as nn
import torch
from transformers import AutoTokenizer, BertModel
import pandas as pd

class CustomBERTModel(nn.Module):
    def __init__(self, bert_embedding_layer, bert_encoder_layers):
        super(CustomBERTModel, self).__init__()
        self.bert_embedding_layer = bert_embedding_layer
        self.bert_encoder_layers = bert_encoder_layers
        
        # Freeze the embedding layer
        for param in self.bert_embedding_layer.parameters():
            param.requires_grad = False

        for param in self.bert_encoder_layers.parameters():
            param.requires_grad = False
        
        ### New layers:
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(768, 256)
        self.linear2 = nn.Linear(256, 1)
        self.linear3 = nn.Linear(512, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, ids, mask):
        embedding_output = self.bert_embedding_layer(ids)
        encoder_outputs = embedding_output
        for i, layer_module in enumerate(self.bert_encoder_layers):
          encoder_outputs = layer_module(encoder_outputs)[0]
        
        linear1_output = self.linear1(encoder_outputs)
        linear2_output = self.linear2(linear1_output)
        flattened_output = self.flatten(linear2_output)
        linear3_output = self.linear3(flattened_output)
        sigmoid_output = self.sigmoid(linear3_output)
        return sigmoid_output


In [10]:
def get_custom_bert_model(num_bert_layers):
    return CustomBERTModel(baseline_model.embeddings, baseline_model.encoder.layer[:num_bert_layers]).to(torch.device("cuda"))

In [11]:
summary(get_custom_bert_model(4))

Layer (type:depth-idx)                             Param #
CustomBERTModel                                    --
├─BertEmbeddings: 1-1                              --
│    └─Embedding: 2-1                              (23,440,896)
│    └─Embedding: 2-2                              (393,216)
│    └─Embedding: 2-3                              (1,536)
│    └─LayerNorm: 2-4                              (1,536)
│    └─Dropout: 2-5                                --
├─ModuleList: 1-2                                  --
│    └─BertLayer: 2-6                              --
│    │    └─BertAttention: 3-1                     (2,363,904)
│    │    └─BertIntermediate: 3-2                  (2,362,368)
│    │    └─BertOutput: 3-3                        (2,361,600)
│    └─BertLayer: 2-7                              --
│    │    └─BertAttention: 3-4                     (2,363,904)
│    │    └─BertIntermediate: 3-5                  (2,362,368)
│    │    └─BertOutput: 3-6                        (2,361,6

In [12]:
# bert_seq_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# print(inputs)
# with torch.no_grad():
#     logits = bert_seq_model(**inputs).logits
#     print(logits.shape)

# predicted_class_id = logits.argmax().item()
# print(bert_seq_model.config.id2label[predicted_class_id])

# summary(bert_seq_model)

In [13]:
inputs = []
labels = []
for idx, row in train_data.iterrows():
    input_text = row['Review']
    inputs.append(input_text)
    labels.append(1 if row['Sentiment'] == 'positive' else 0)


In [14]:
import torch
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label)
        }

In [15]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
dataset = TextDataset(inputs, labels, tokenizer, 512)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [16]:
def train_custom_model(model, epochs, dataloader):
    criterion = nn.BCELoss() ## If required define your own criterion
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
    for epoch in range(epochs):
        for batch in dataloader: 
            targets = np.array(batch['label'])
            targets = torch.tensor(np.expand_dims(targets,axis=1)).float().cuda()
            optimizer.zero_grad()
            input_ids = batch['input_ids'].cuda()
            attention_mask = batch['attention_mask'].cuda()
            outputs = model(input_ids.cuda(), attention_mask.cuda())
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        print("Epoch: " + str(epoch) + " Loss: " + str(loss))

In [17]:
def predict_custom(model, df):
    model.eval()
    predictions = []
    labels = []
    with torch.no_grad():
        for idx, row in df.iterrows():
            input_text = row['Review']
            encoding = tokenizer(input_text, add_special_tokens=True, max_length = 512, padding='max_length', truncation=True, return_tensors='pt')
            input_ids = encoding['input_ids'].cuda()
            attention_mask = encoding['attention_mask'].cuda()
            prediction = model(encoding['input_ids'].cuda(), encoding['attention_mask'].cuda())
            predictions.append(1 if prediction > 0.5 else 0)
            labels.append(convert_lbl_to_int(row['Sentiment']))

    return predictions, labels

In [18]:
def score_custom_model(model, train_df, test_df):
    train_predictions, train_labels = predict_custom(model, train_df)
    test_predictions, test_labels = predict_custom(model, test_df)
    return f1_score(train_labels, train_predictions, average='micro'), f1_score(test_labels, test_predictions, average='micro')
    
# print(score_custom_model(model, train_data, test_data))

In [19]:
def score_all_probe_models(dataloader, train_df, test_df, epochs):
    model_scores = []
    for n in range(1,12):
        print("N: " + str(n))
        model = get_custom_bert_model(n)
        train_custom_model(model, epochs, dataloader)
        train_f1, test_f1 = score_custom_model(model, train_df, test_df)
        model_scores.append([n, train_f1, test_f1])

    return model_scores        

In [20]:
model_scores = score_all_probe_models(dataloader, train_data, test_data, 5)

N: 1
Epoch: 0 Loss: tensor(0.6710, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 1 Loss: tensor(0.4555, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 2 Loss: tensor(0.4637, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 3 Loss: tensor(0.3358, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 4 Loss: tensor(0.4363, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
N: 2
Epoch: 0 Loss: tensor(0.5693, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 1 Loss: tensor(0.4542, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 2 Loss: tensor(0.4242, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 3 Loss: tensor(0.4298, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 4 Loss: tensor(0.5534, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
N: 3
Epoch: 0 Loss: tensor(0.6479, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Epoch: 1 Loss: tensor(0.5939, device='c

In [21]:
print(model_scores)

[[1, 0.845125, 0.8045], [2, 0.80125, 0.7555], [3, 0.79675, 0.7605], [4, 0.83325, 0.7925], [5, 0.817125, 0.783], [6, 0.820625, 0.787], [7, 0.823625, 0.8015], [8, 0.86125, 0.8125], [9, 0.87175, 0.846], [10, 0.88475, 0.8555], [11, 0.889875, 0.856]]


## Analysis

In [182]:
summary(baseline_model)

Layer (type:depth-idx)                                  Param #
BertModel                                               --
├─BertEmbeddings: 1-1                                   --
│    └─Embedding: 2-1                                   23,440,896
│    └─Embedding: 2-2                                   393,216
│    └─Embedding: 2-3                                   1,536
│    └─LayerNorm: 2-4                                   1,536
│    └─Dropout: 2-5                                     --
├─BertEncoder: 1-2                                      --
│    └─ModuleList: 2-6                                  --
│    │    └─BertLayer: 3-1                              7,087,872
│    │    └─BertLayer: 3-2                              7,087,872
│    │    └─BertLayer: 3-3                              7,087,872
│    │    └─BertLayer: 3-4                              7,087,872
│    │    └─BertLayer: 3-5                              7,087,872
│    │    └─BertLayer: 3-6                              

In [243]:
summary(model, [1, 47])

Layer (type:depth-idx)                             Output Shape              Param #
CustomBERTModel                                    [64, 1]                   --
├─BertEmbeddings: 1-1                              [64, 512, 768]            --
│    └─Embedding: 2-1                              [64, 512, 768]            23,440,896
│    └─Embedding: 2-2                              [64, 512, 768]            1,536
│    └─Embedding: 2-3                              [1, 512, 768]             393,216
│    └─LayerNorm: 2-4                              [64, 512, 768]            1,536
│    └─Dropout: 2-5                                [64, 512, 768]            --
├─ModuleList: 1-2                                  --                        --
│    └─BertLayer: 2-6                              [64, 512, 768]            --
│    │    └─BertAttention: 3-1                     [64, 512, 768]            2,363,904
│    │    └─BertIntermediate: 3-2                  [64, 512, 3072]           2,362,368
│ 